# 0. Install Dependencies

In [ ]:
!apt update && apt install xvfb
!pip install gym-notebook-wrapper
!apt update && apt install python-opengl ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
!pip install tensorflow gym keras-rl2 gym[atari]
!pip install ale_py


     |████████████████████████████████| 52 kB 548 kB/s 
     |████████████████████████████████| 1.6 MB 5.2 MB/s 


# 1. Test Random Environment with OpenAI Gym

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/RL project/roms/space_invaders.bin' '/usr/local/lib/python3.7/dist-packages/ale_py/roms/'

In [ ]:
import gym 
import random
from ale_py import ALEInterface
from ale_py.roms import SpaceInvaders
import gnwrapper

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [ ]:
ale = ALEInterface()
env = gnwrapper.Monitor(gym.make('SpaceInvaders-v0'), directory= "./", force=True)
height, width, channels = env.observation_space.shape
actions = env.action_space.n 

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
actions = 6

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP',
 'FIRE',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT',
 'UPFIRE',
 'RIGHTFIRE',
 'LEFTFIRE',
 'DOWNFIRE',
 'UPRIGHTFIRE',
 'UPLEFTFIRE',
 'DOWNRIGHTFIRE',
 'DOWNLEFTFIRE']

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
# env.close()
env.display()

Episode:1 Score:75.0
Episode:2 Score:155.0
Episode:3 Score:90.0
Episode:4 Score:120.0
Episode:5 Score:80.0


'openaigym.video.1.73.video000000.mp4'

'openaigym.video.1.73.video000001.mp4'

# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
# del model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_1 (Flatten)         (None, 67584)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               34603520  
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 6)                

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  358/10000: episode: 1, duration: 14.333s, episode steps: 358, steps per second:  25, episode reward: 15.000, mean reward:  0.042 [ 0.000, 10.000], mean action: 2.335 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  721/10000: episode: 2, duration: 5.916s, episode steps: 363, steps per second:  61, episode reward: 105.000, mean reward:  0.289 [ 0.000, 25.000], mean action: 2.264 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1027/10000: episode: 3, duration: 32.875s, episode steps: 306, steps per second:   9, episode reward: 50.000, mean reward:  0.163 [ 0.000, 20.000], mean action: 2.493 [0.000, 5.000],  loss: 121.960197, mean_q: 6.378694, mean_eps: 0.908785
 1509/10000: episode: 4, duration: 474.538s, episode steps: 482, steps per second:   1, episode reward: 180.000, mean reward:  0.373 [ 0.000, 30.000], mean action: 2.434 [0.000, 5.000],  loss: 4.523840, mean_q: 1.877863, mean_eps: 0.885925
 1941/10000: episode: 5, duration: 412.163s, episode steps: 432, steps per second:   1, episode reward: 65.000, mean reward:  0.150 [ 0.000, 20.000], mean action: 2.484 [0.000, 5.000],  loss: 3.677307, mean_q: 2.236436, mean_eps: 0.844795
 2324/10000: episode: 6, duration: 360.129s, episode steps: 383, steps per second:   1, episode reward: 75.000, mean reward:  0.196 [ 0.000, 25.000], mean action: 2.507 [0.000, 5.000],  loss: 2.844665, mean_q: 1.922221, mean_eps: 0.808120
 2750/10000: episode: 7, duration: 398.880

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Episode 1: reward: 120.000, steps: 394
Episode 2: reward: 20.000, steps: 401
Episode 3: reward: 5.000, steps: 226
Episode 4: reward: 105.000, steps: 397
Episode 5: reward: 50.000, steps: 292
Episode 6: reward: 50.000, steps: 390
Episode 7: reward: 0.000, steps: 194
Episode 8: reward: 60.000, steps: 228
Episode 9: reward: 20.000, steps: 303
Episode 10: reward: 0.000, steps: 193
43.0


# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('/content/drive/MyDrive/Colab Notebooks/RL project/SavedWeights/10k-Colab/dqn_weights-self.h5f')

In [ ]:
del model, dqn

In [ ]:
# dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')
dqn.load_weights('/content/drive/MyDrive/Colab Notebooks/RL project/SavedWeights/1m/dqn_weights.h5f')